In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id':'11y79Tmi7znU_ancRPVVn5MWffJ1i_9G0'}) # replace the id with id of file that you want to access
downloaded.GetContentFile('data.csv')

In [0]:
import sklearn
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn import metrics
from numpy import array
from numpy import mean
from numpy import cov
from numpy.linalg import eig
from copy import deepcopy

In [0]:
def Accuracy(y_true,y_pred):
    """
    :type y_true: numpy.ndarray
    :type y_pred: numpy.ndarray
    :rtype: float
    
    """
    
    rightly_pred = 0
    for i in range(len(y_true)):
      if y_true[i]==y_pred[i]:
        rightly_pred = rightly_pred+1
    accuracy = rightly_pred/(len(y_true))
    return accuracy


In [0]:
def Recall(y_true,y_pred):
    """
    :type y_true: numpy.ndarray
    :type y_pred: numpy.ndarray
    :rtype: float
    """
    #using confusion matrix to calculate recall
    x = ConfusionMatrix(y_true,y_pred)
    diag = np.diagonal(x)
    rec = 0
    for i in range(len(diag)):
      if diag[i] == 0:
        rec = rec + 0
      else:
        rec = rec + (diag[i])/(np.sum(x[i]))
    recall = rec / len(diag)
    return recall

In [0]:
def Precision(y_true,y_pred):
    """
    :type y_true: numpy.ndarray
    :type y_pred: numpy.ndarray
    :rtype: float
    """
    #Using confusion matrix to calculate precision
    x = ConfusionMatrix(y_true,y_pred)
    diag = np.diagonal(x)
    prec = 0
    for i in range(len(diag)):
      if diag[i] == 0:
        prec = prec + 0
      else:
        prec = prec + (diag[i])/(np.sum(x[:,i]))
    precision = prec / len(diag)
    return precision

In [0]:
def ConfusionMatrix(y_true,y_pred):
    
    """
    :type y_true: numpy.ndarray
    :type y_pred: numpy.ndarray
    :rtype: float
    """  
    #y_true is a list of list after undergoing split function.  
    y_true = [i[0] for i in y_true]
    number_of_classes = len(np.unique(y_true))
    inter_res_1 = np.add(y_pred,(np.multiply(y_true, number_of_classes)))
    inter_res_2 = np.histogram(inter_res_1, bins=np.arange(number_of_classes+1, pow(number_of_classes,2)+number_of_classes+2))
    Conf_matrix = inter_res_2[0].reshape(number_of_classes,number_of_classes)
    return Conf_matrix

In [0]:
def normalize2(X):
  import sklearn
  from sklearn.preprocessing import MinMaxScaler
  scaler = MinMaxScaler()
  scaler.fit(X)
  norm_X = scaler.transform(X)
  return norm_X


def normalize1(X_train):
  #MinMaxScaler
  print(X_train)
  for data in X_train:
    min_val = np.min(X_train[data])
    #print(min_val)
    max_val = np.max(X_train[data])
    X[data] = (X_train[data]-min_val)/(max_val-min_val)
  X=X.to_numpy()
  return X

def normalize_preprocess1(X_train, X_test):
  #using np.split
  Combined = np.append(X_train, X_test, axis = 0)
  #print(Combined.type)
  Combined = normalize2(Combined)
  X_train1 = np.split(Combined,[0,X_train.shape[0]], axis = 0)
  X_train = X_train1[1]
  X_test = X_train1[2]
  return X_train,X_test


def Preprocessing(X_train, X_test, Y_train, Y_test):
    X_train = X_train.to_numpy()
    X_test = X_test.to_numpy()
    Y_train = Y_train.to_numpy()
    Y_test = Y_test.to_numpy()
    return X_train, X_test, Y_train, Y_test

In [0]:
#function that calculates euclidean distance using tile
def euclidean(X_train,center):
    z = pd.np.tile(center,(len(X_train),1))
    ans = np.subtract(X_train,z)
    ans = pow(ans,2)
    ans = np.sum(ans, axis = 1)
    ans = np.sqrt(ans)
    return ans

In [0]:
#K-NEAREST NEIGHBOR FUNCTION
def KNN(X_train,X_test,Y_train,K):
    """
    :type X_train: numpy.ndarray
    :type X_test: numpy.ndarray
    :type Y_train: numpy.ndarray
    
    :rtype: numpy.ndarray
    """
    X_train, X_test = normalize_preprocess1(X_train, X_test)
    i=0
    y_pred = []
    correct = 0
    m = len(X_train)
    for test_sample in X_test:
      ans = euclidean(X_train,test_sample)
      index_neighbour = np.argpartition(ans, K)[:K]
      pred = []
      for ind in index_neighbour:
        pred.append(Y_train[ind][0])
      pred_val =  max(pred, key = pred.count)
      y_pred.append(pred_val)
    return y_pred

In [0]:
#PCA IMPLEMENTATION
def PCA(X_train,N):
    """
    :type X_train: numpy.ndarray
    :type N: int
    :rtype: numpy.ndarray
    """
    X_train = normalize2(X_train)
    M = mean(X_train.T, axis = 1)
    C = X_train - M
    V = cov(C.T)
    eig_val_cov, eig_vec_cov = np.linalg.eig(V)
    eig_pairs = [(np.abs(eig_val_cov[i]), eig_vec_cov[:,i]) for i in range(len(eig_val_cov))]
    eig_pairs.sort(key=lambda x: x[0], reverse=True)
    W = []
    for n in eig_pairs[0][1].tolist():
        W.append([n])

    for i in range(1, N):
        npstack(W, eig_pairs[i][1].tolist())

    T = X_train.dot(W)

    return T

In [0]:
def WCSS(clustered_data):
    sq_sum = 0
    centers = []
    for i in range(k):
        centers.append(np.mean(clustered_data[i],axis = 0)) 
    for i in range(k):
        sum1 = euclidean(clustered_data[i], centers[i])
        sq_sum = sq_sum + np.sum(sum1,axis = 0)
    return sq_sum  

In [0]:
#KMEANS IMPLEMENTATION
def Kmeans(X_train, X_test, Y_train, k):
    """
    :type X_train: numpy.ndarray
    :type N: int
    :rtype: List[numpy.ndarray]
    """
    X_train, X_test = normalize_preprocess1(X_train, X_test)
    n=X_train.shape[0] #number of training examples
    c=X_train.shape[1] #number of features
    mean = np.mean(X_train, axis = 0)
    std = np.std(X_train, axis = 0)
    centers = np.random.randn(k,c)*std + mean #Generate random centers
    centers_old = np.ones(centers.shape)
    centers_new = deepcopy(centers) # Store new centers
    clusters = np.ones(n)
    distances = np.ones((n,k))
    chistances = np.ones((n,k))
    error = np.linalg.norm(centers_new - centers_old)

    while error != 0:
        for i in range(k):
            distances[:,i] = euclidean(X_train,centers_new[i])
        clusters = np.argmin(distances, axis = 1)
        centers_old = deepcopy(centers_new)
        # Calculate mean for every cluster and update the center
        for i in range(k):
            if (len(X_train[clusters == i]) != 0):
                centers_new[i] = np.mean(X_train[clusters == i], axis=0)
        error = np.linalg.norm(centers_new - centers_old)
    for i in range (k):
      clustered_data.append(data[clusters == i])    
    return clustered_data   


In [0]:
def SklearnSupervisedLearning(X_train,Y_train,X_test, Y_test):
    """
    :type X_train: numpy.ndarray
    :type X_test: numpy.ndarray
    :type Y_train: numpy.ndarray
    
    :rtype: List[numpy.ndarray] 
    """

    #Added Y_test to the parameters
    X_train, X_test = normalize_preprocess1(X_train, X_test)
    clf_knn = KNeighborsClassifier(n_neighbors=7)
    clf_knn.fit(X_train, Y_train)
    y_pred=clf_knn.predict(X_test)
    print("Accuracy of KNN: ", Accuracy(Y_test,y_pred))


    clf = LogisticRegression(random_state=0).fit(X_train, Y_train)
    y_pred_log = clf.predict(X_test)
    print("Accuracy of Logistic Regression: ", Accuracy(Y_test,y_pred_log))
  
    clf1 = DecisionTreeClassifier(random_state=0).fit(X_train,Y_train)
    y_pred_dec = clf1.predict(X_test)
    print("Accuracy of Decision Tree Classifier: ", Accuracy(Y_test,y_pred_dec))

    clf2 = SVC(gamma='auto').fit(X_train,Y_train)
    y_pred_svc = clf2.predict(X_test)
    print("Accuracy of Support Vector Classifier: ", Accuracy(Y_test,y_pred_dec))

    #Plots for confusion matrix
    figure, [[ax1, ax2], [ax3, ax4]] = plt.subplots(2,2)
    figure.set_figheight(20)
    figure.set_figwidth(20)
    
    
    conf_knn = ConfusionMatrix(Y_test,y_pred)
    ax1.imshow(conf_knn, cmap = 'Blues')
    for (i, j), z in np.ndenumerate(conf_knn):
      ax1.annotate(str(z),xy=(j,i), ha='center', va='center')
    
    ax1.set_xticks(np.arange(1, 11))
    ax1.set_yticks(np.arange(1, 11))
    ax1.set_xlabel('True Label')
    ax1.set_ylabel('Predicted Label')
    ax1.set_title('Confusion Matrix of KNN')


    conf_log = ConfusionMatrix(Y_test,y_pred_log)
    ax2.matshow(conf_log, cmap = 'Blues')
    for (i1, j1), z2 in np.ndenumerate(conf_log):
      ax2.annotate(str(z2),xy=(j1,i1), ha='center', va='center')
    ax2.set_xticks(np.arange(1, 11))
    ax2.set_yticks(np.arange(1, 11))
    ax2.set_xlabel('True Label')
    ax2.set_ylabel('Predicted Label')
    ax2.set_title('Confusion Matrix of Logistic Regression')
    
    
    conf_dtc = ConfusionMatrix(Y_test,y_pred_dec)
    ax3.matshow(conf_dtc, cmap = 'Blues')
    for (i3, j3), z3 in np.ndenumerate(conf_dtc):
      ax3.annotate(str(z3),xy=(j3,i3), ha='center', va='center')
    ax3.set_xticks(np.arange(1, 11))
    ax3.set_yticks(np.arange(1, 11))
    ax3.set_xlabel('True Label')
    ax3.set_ylabel('Predicted Label')
    ax3.set_title('Confusion Matrix of Decision Tree')
    
    
    conf_svm = ConfusionMatrix(Y_test,y_pred_svc)
    ax4.matshow(conf_svm, cmap = 'Blues')
    for (i4, j4), z4 in np.ndenumerate(conf_svm):
      ax4.annotate(str(z4),xy=(j4,i4), ha='center', va='center')
    ax4.set_xticks(np.arange(1, 11))
    ax4.set_yticks(np.arange(1, 11))
    ax4.set_xlabel('True Label')
    ax4.set_ylabel('Predicted Label')
    ax4.set_title('Confusion Matrix of SVM')
    plt.show

    lst_predictions = [y_pred, y_pred_log, y_pred_dec, y_pred_svc]
    return lst_predictions

In [0]:
def SklearnVotingClassifier(X_train,Y_train,X_test, Y_test):
    
    """
    :type X_train: numpy.ndarray
    :type X_test: numpy.ndarray
    :type Y_train: numpy.ndarray
    
    :rtype: List[numpy.ndarray] 
    """
    X_train, X_test = normalize_preprocess1(X_train, X_test)
    Y_train = Y_train.reshape(Y_train.shape[0],)
    Y_test = Y_test.reshape(Y_test.shape[0],)
    X_scaled = preprocessing.scale(X_train)
    lst_of_classifiers= []
    clf_knn = KNeighborsClassifier(n_neighbors=7)
    lst_of_classifiers.append(('knn',clf_knn))
    clf = LogisticRegression(random_state=0)
    lst_of_classifiers.append(('Logistic', clf))
    clf1 = DecisionTreeClassifier(random_state=0)
    lst_of_classifiers.append(('Decision', clf1))
    clf2 = SVC(gamma='auto')
    lst_of_classifiers.append(('SVM', clf2)) 
    ensemble_mod = VotingClassifier(lst_of_classifiers)
    classifier3 = ensemble_mod.fit(X_train, Y_train)
    y_pred = classifier3.predict(X_test)
    print("Accuracy of ensemble model: ", Accuracy(Y_test,y_pred))
    return y_pred

In [0]:
#Hyperparameters : Number of neighbours, metric used for calculation of distance
def GridSearchKnn(X_train,Y_train):
  X_train = normalize2(X_train)
  neighbours = [3, 5, 7]
  grid_params = {'n_neighbors' : neighbours, 'metric' : ['euclidean','manhattan']}
  Y_train = Y_train.reshape(Y_train.shape[0],)
  clf = GridSearchCV(KNeighborsClassifier(), grid_params, scoring='accuracy')
  model = clf.fit(X_train, Y_train)
  print('Best number of neighbours for KNN: ',model.best_estimator_.get_params()['n_neighbors'])
  print('Best distance for KNN: ',model.best_estimator_.get_params()['metric'])
  pvt = pd.pivot_table(pd.DataFrame(model.cv_results_),values='mean_test_score', index = 'param_n_neighbors', columns = 'param_metric')
  #pvt = pd.pivot_table(pd.DataFrame(model.cv_results_),columns = 'param_metric')  
  ax = sns.heatmap(pvt,annot=True)
  ax1 = pvt.plot()
  ax1.set_xlabel("Number of Neighbors")
  ax1.set_ylabel("Accuracy")
  return pvt

In [0]:
#Hypeparameters : Depth, impurity calculation method
def GridSearchDTC(X_train, Y_train):
  X_train = normalize2(X_train)
  depth = np.arange(8,12)
  #params_dtc = {'max_depth': depth}
  params_dtc = {'max_depth': depth,'criterion':['gini','entropy']}
  clf_dtc = GridSearchCV(DecisionTreeClassifier(), params_dtc, scoring='accuracy')
  model_dtc = clf_dtc.fit(X_train, Y_train)
  print('Maximum Depth: ',model_dtc.best_estimator_.get_params()['max_depth'])
  print('Criterion to calculate impurity:',model_dtc.best_estimator_.get_params()['criterion'])
  pvt = pd.pivot_table(pd.DataFrame(model_dtc.cv_results_),values='mean_test_score', index = 'param_max_depth', columns = 'param_criterion')
  ax = sns.heatmap(pvt,annot=True)
  ax1 = pvt.plot()
  ax1.set_xlabel("Depth")
  ax1.set_ylabel("Accuracy")
  return pvt 

In [0]:
#Hyperparameters : Regularization parameter 
def GridSearchSvm(X_train,Y_train):
  X_train = normalize2(X_train)
  regularization = [0.1,0.5,1,10]
  Y_train = Y_train.reshape(Y_train.shape[0],)
  grid_params_svm = {'C' : regularization}
  clf_svm = GridSearchCV(svm.SVC(kernel='rbf'), grid_params_svm, scoring='accuracy')
  model_svm = clf_svm.fit(X_train, Y_train)
  print('Best regularization for SVM',model_svm.best_estimator_.get_params()['C'])
  lst_mean_scores_svm = model_svm.cv_results_['mean_test_score']
  plt.plot(regularization, lst_mean_scores_svm, linestyle='solid')
  plt.xlabel('Regularization')
  plt.ylabel('Accuracy')
  plt.show()

In [0]:
#We have used MinMax scaling as the models perform better under this scaling. 
#Therefore, the normalize function is called within the functions and the data needn't be normalized before being fed into the functions. 
dfs = pd.read_csv('data.csv',encoding='unicode_escape')
X = dfs.iloc[:,:48]
y = dfs.iloc[:,48:]

X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.2)
X_train, X_test, Y_train, Y_test = Preprocessing(X_train,X_test,Y_train,Y_test) 
X_train

PCA(X_train,3)

y_pred = KNN(X_train,X_test,Y_train, 5)
print(ConfusionMatrix(Y_test, y_pred))
print(Accuracy(Y_test,y_pred))
print(Recall(Y_test,y_pred))
print(Precision(Y_test,y_pred))

Kmeans(X_train, X_test, Y_train, 11)


SklearnSupervisedLearning(X_train,Y_train,X_test, Y_test)

SklearnVotingClassifier(X_train,Y_train,X_test, Y_test)

GridSearchKnn(X_train,Y_train)

GridSearchDTC(X_train,Y_train)

GridSearchSvm(X_train,Y_train)

#K-Nearest Neighbour implementation
y_pred = KNN(X_train,X_test,Y_train, 5)
print(ConfusionMatrix(Y_test, y_pred))
print(Accuracy(Y_test,y_pred))
print(Recall(Y_test,y_pred))
print(Precision(Y_test,y_pred))

#X_train1 = normalize1(X_train)
#shape of training data
m = X_train.shape
print(X_train)
print(m[0])
n = X_test.shape
print(n[0])


Combined = np.append(X_train, X_test, axis = 0)
Combined = normalize1(Combined)
X_train1 = np.split(Combined,[0,X_train.shape[0]], axis = 0)
X_train = X_train1[1]
X_test = X_train1[2]

print("New shape: ", X_train.shape)
print("New shape: ", X_test.shape)